# Exercise 2: Conduction and Convection with RC-Networks

To practice the application of the numerical methods discussed in the lecture, you will write a small script to solve the temperature for a given RC-Network with conduction and convection.

The following example of an RC-Network as depicted in Figure 1 should be given. The network consists of resistances and conductances with non capacitive nodes.

```{figure} Ex2/ex2_rc_network.png
---
height: 300px
name: ex2_rc_network
---
Figure from problem statement. {cite}`hagentoftIntroductionBuildingPhysics2001`, {cite}`claessonNetworkAnalysisNumerical1994`.
```

The problem should be solved using Equation